# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI. Отдельно проанализируйте 3 ошибочно кластеризованных текста (если такие есть).

In [ ]:
%pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=c5f3c4c5eeded7598458f4ca7ca111809a14d9ff171ceb944b298f4ab60d6dba
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import wikipedia
import numpy as np

In [ ]:
langs = ['de', 'no', 'da', 'lb' , 'is']

In [ ]:
def load_with_disambigution(page):
    try:
        p = wikipedia.page(page)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        p = wikipedia.page(random_option)
    return p

In [ ]:
def get_texts_for_lang(lang, n=200): # функция для скачивания статей из википедии
    # вот так можно зафиксировать язык
    wikipedia.set_lang(lang)
    wiki_content = []

    # random достает только ссылки на статьи
    pages = wikipedia.random(n)

    for page_name in pages:
        try:
            # чтобы загрузить контент статьи можно вызвать функцию .page
            page = load_with_disambigution(page_name)

        except Exception as e:
            print('Skipping page {}'.format(page_name), str(e).strip('\n'))
            continue

        wiki_content.append(f'{page.title}\n{page.content.replace("==", "")}')

    return wiki_content

In [ ]:
%%time

wiki_texts = {}

for lang in langs:
    try:
        wiki_texts[lang] = get_texts_for_lang(lang, 10)
    except Exception as e:
        print('ERROR ON - ', lang, str(e).strip('\n'))
        continue

    print(lang, len(wiki_texts[lang]))

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


de 10
no 10
da 10
lb 10
is 10
CPU times: user 2.21 s, sys: 167 ms, total: 2.38 s
Wall time: 38.4 s


In [ ]:
wiki_texts['de'][1]

'Anne Shilcock\nAnne Shilcock (* 13. Juni 1932 in Hartfield, England; † April 2019 in Südafrika) war eine britische Tennisspielerin aus England.\n\n\n Erfolge \n1955 gewann Shilcock, die ihre aktive Zeit in den 1950er Jahren hatte, mit ihrer Landsfrau Angela Mortimer in Wimbledon die Konkurrenz im Damendoppel. Sie besiegten Shirley Bloomer und Patricia Ward Hales im Endspiel mit 7:5, 6:1.\n\n\n Weblinks \nAnne Shilcock auf www.tennis-spieler.com\n\n\n Einzelnachweise '

Preprocessing:

In [ ]:
!pip install nltk

In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def preprocess(text):
    """
    Функция для очистки текста от ненужных тегов, разделителей и отступов.
    """
    # Удаляем заголовки и подзаголовки (== Заголовок == или === Подзаголовок ===)
    text = re.sub(r'={2,}.*?={2,}', '', text)
    # Удаляем HTML и вики-теги (например, <ref>...</ref>, <sup>...</sup>)
    text = re.sub(r'<.*?>', '', text)
    # Удаляем шаблоны в формате {{Шаблон}}
    text = re.sub(r'\{\{.*?\}\}', '', text)
    # Удаляем квадратные скобки [[Link]] или [[Link|display text]]
    text = re.sub(r'\[\[.*?\|?(.*?)\]\]', r'\1', text)
    # Убираем лишние пробелы и переносы строк
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

Разбиение на предложения:

In [ ]:
def split_into_sentences(text, lang):
    # Определяем язык для токенизации предложений в nltk
    lang_map = {
        'de': 'german',
        'no': 'norwegian',
        'da': 'danish',
        'lb': 'german',  # Люксембургский не поддерживается nltk, используем близкий немецкий
        'is': 'english'  # Исландский также не поддерживается, приближенно используем английский
    }
    sentences = nltk.sent_tokenize(text, language=lang_map.get(lang, 'english'))
    return sentences

Dataset:

In [ ]:
preprocessed_data = []

for lang, texts in wiki_texts.items():
    for text in texts:
        # Чистим текст
        cleaned_text = preprocess(text)
        # Разбиваем на предложения
        sentences = split_into_sentences(cleaned_text, lang)
        for sentence in sentences:
            preprocessed_data.append({'language': lang, 'sentence': sentence})

In [ ]:
import pandas as pd

In [ ]:
# Создание DataFrame и сохранение в CSV
df = pd.DataFrame(preprocessed_data)
df.to_csv('preprocessed_wiki_sentences.csv', index=False, encoding='utf-8')
print("Processed dataset saved to 'processed_wiki_sentences.csv'")

Processed dataset saved to 'processed_wiki_sentences.csv'


In [ ]:
df

,language,sentence
0,de,Proč Proč (ungarisch Porócs – bis 1907 Prócs) ...
1,de,Geographie Die Gemeinde befindet sich am Südha...
2,de,Nordwestlich des Ortes erhebt sich der 604 m n.m.
3,de,hohe Berg Haľagoš.
4,de,Das Ortszentrum liegt auf einer Höhe von 467 m...
...,...,...
881,is,RS:X RS:X er seglbretti framleitt af Neil Pryde.
882,is,Það var valið sem ólympíubretti af Alþjóðlegu ...
883,is,Það tók við af Mistral One Design sem var ólym...
884,is,RS:X er framleitt með tvenns konar búnaði þar ...


**КЛАСТЕРИЗАЦИЯ**

In [ ]:
grouped_df = df.groupby('language')[['language', 'sentence']]

In [ ]:
for key, _ in grouped_df:
    print(grouped_df.get_group(key), "\n\n")
    break

    language                                           sentence
508       da  Bent Melchior Bent Melchior (født 24. juni 192...
509       da  Baggrund og levned Melchior blev født i den da...
510       da  Marcus Melchior var på daværende tidspunkt rab...
511       da  Bent fik i alt fem søskende, heriblandt den da...
512       da  I 1933 vendte familien tilbage til Danmark, me...
..       ...                                                ...
731       da                Alle gårdens 5 Længer brændte 1709.
732       da                 Det nuværende stuehus er fra 1915.
733       da  I 1925 blev der udstykket 14 statshusmandsbrug...
734       da  Eksterne kilder og henvisninger Om Slumstrup i...
735       da  Trap: Kongeriget Danmark, udarbejdet af H. Wei...

[228 rows x 2 columns] 




In [ ]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering, OPTICS
import numpy as np
from sklearn.metrics import adjusted_rand_score

from IPython.display import Image
from IPython.core.display import HTML

In [ ]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['sentence'].values

    # создаем пустую матрицу для векторов
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    cluster = AgglomerativeClustering(n_clusters=1)

    cluster.fit(X)
    labels = np.array(cluster.labels_)+1

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['language'], labels))

print("Метрика ARI:", np.mean(ARI)) # усредненная метрика

Метрика ARI: 1.0


In [ ]:

# Метрика ARI для оценки кластеров
ARI = []

# Перебор по каждому языку в `grouped_df`
for key, _ in grouped_df:
    # Извлекаем предложения для текущего языка
    texts = grouped_df.get_group(key)['sentence'].values

    # Преобразуем предложения в векторы
    X = embed(texts)

    # Кластеризация предложений
    cluster = AgglomerativeClustering(n_clusters=1)  # предполагаем один кластер для каждого языка
    cluster.fit(X)
    labels = np.array(cluster.labels_)

    # Поскольку для каждой группы существует один язык, мы создадим массив истинных меток:
    true_labels = np.zeros(len(labels))  # все метки 0, так как один кластер

    # Вычисляем ARI, который будет равен 1.0, если все предложения собраны в один кластер
    ARI.append(adjusted_rand_score(true_labels, labels))

# Усредненная метрика ARI
print("Средняя метрика ARI:", np.mean(ARI))

Средняя метрика ARI: 1.0


In [ ]:
# Создаём числовые метки для языков для вычисления ARI
language_labels, label_to_language = pd.factorize(df['language'])
df['label'] = language_labels

# Преобразуем тексты в эмбеддинги
embeddings = embed(df['sentence'].tolist())

# Задаём количество кластеров равным числу уникальных языков
n_clusters = len(label_to_language)

# Инициализируем и обучаем модель кластеризации на всех эмбеддингах
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
predicted_labels = kmeans.fit_predict(embeddings)

# Добавляем предсказанные метки к DataFrame для анализа
df['predicted_label'] = predicted_labels

# Вычисляем ARI для всей выборки
ari_score = adjusted_rand_score(df['label'], predicted_labels)
print("Метрика ARI:", ari_score)

# Находим ошибочно кластеризованные тексты
misclassified_texts = df[df['label'] != df['predicted_label']]

# Проверяем наличие ошибочных кластеризаций и выводим первые три для анализа
if not misclassified_texts.empty:
    print("\nОшибочно кластеризованные тексты:")
    for i, row in misclassified_texts.head(3).iterrows():
        print(f"\nТекст: {row['sentence']}")
        print(f"Язык (истинный): {label_to_language[row['label']]}")
        print(f"Кластер (предсказанный): {label_to_language[row['predicted_label']]}")
else:
    print("\nОшибочно кластеризованных текстов нет.")

Метрика ARI: 0.39597204845785877

Ошибочно кластеризованные тексты:

Текст: Proč Proč (ungarisch Porócs – bis 1907 Prócs) ist eine Gemeinde im Osten der Slowakei mit 416 Einwohnern (Stand 31. Dezember 2022) im Okres Prešov, einem Teil des Prešovský kraj, und wird zur traditionellen Landschaft Šariš gezählt.
Язык (истинный): de
Кластер (предсказанный): is

Текст: Geographie Die Gemeinde befindet sich am Südhang der Niederen Beskiden am Oberlauf der Ladianka im Einzugsgebiet des Sekčov und weiter der Torysa.
Язык (истинный): de
Кластер (предсказанный): is

Текст: Nordwestlich des Ortes erhebt sich der 604 m n.m.
Язык (истинный): de
Кластер (предсказанный): lb


# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор в разных предложениях). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

In [ ]:
!wget https://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2

--2024-11-07 14:09:27--  https://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2
Resolving opencorpora.org (opencorpora.org)... 172.67.163.210, 104.21.15.199, 2606:4700:3031::ac43:a3d2, ...
Connecting to opencorpora.org (opencorpora.org)|172.67.163.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1435269 (1.4M) [application/x-bzip2]
Saving to: ‘annot.opcorpora.no_ambig_strict.xml.bz2’

annot.opcorpora.no_ 100%[===================>]   1.37M  1.99MB/s    in 0.7s    

2024-11-07 14:09:29 (1.99 MB/s) - ‘annot.opcorpora.no_ambig_strict.xml.bz2’ saved [1435269/1435269]



In [ ]:
import bz2

with bz2.open('annot.opcorpora.no_ambig_strict.xml.bz2', 'rb') as f_in, open('annot.opcorpora.no_ambig_strict.xml', 'wb') as f_out:
    f_out.write(f_in.read())

In [ ]:
%pip install lxml

In [ ]:
from lxml import etree

In [ ]:
from collections import defaultdict, Counter

In [ ]:
open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

In [ ]:
# Список для хранения предложений с токенами и их грамматическими разборами
corpus = []

for sentence in open_corpora.xpath('//tokens'):
    sent_tagged = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')[0]
        gram_info = tuple(token.xpath('tfr/v/l/g/@v'))  # Сохраняем разбор как кортеж для уникальности
        sent_tagged.append((word, gram_info))
    corpus.append(sent_tagged)

In [ ]:
# Словарь для хранения уникальных разборов каждой словоформы
word_forms = defaultdict(set)

# Заполняем словарь word_forms
for sentence in corpus:
    for word, gram_info in sentence:
        word_forms[word].add(gram_info)

# Находим омонимичные словоформы (с несколькими разными разборами)
homonym_forms = {form: analyses for form, analyses in word_forms.items() if len(analyses) > 1}

# Считаем частоту омонимичных словоформ
homonym_counter = Counter({form: len(analyses) for form, analyses in homonym_forms.items()})
most_frequent_homonyms = homonym_counter.most_common()

# Находим словоформы с максимальным количеством вариантов грамматического разбора
max_analysis_forms = sorted(homonym_forms.items(), key=lambda item: len(item[1]), reverse=True)

In [ ]:
# Вывод результатов
print("Самые частотные омонимичные словоформы:")
for form, count in most_frequent_homonyms[:10]:  # Показываем топ-10
    print(f"{form}: {count} вариантов разбора")

print("\nСловоформы с максимальным количеством грамматических разборов:")
for form, analyses in max_analysis_forms[:10]:  # Показываем топ-10
    print(f"{form}: {len(analyses)} вариантов разбора")

Самые частотные омонимичные словоформы:
США: 6 вариантов разбора
кино: 5 вариантов разбора
евро: 5 вариантов разбора
компании: 5 вариантов разбора
какой: 5 вариантов разбора
одной: 4 вариантов разбора
своей: 4 вариантов разбора
лица: 4 вариантов разбора
этой: 4 вариантов разбора
версии: 4 вариантов разбора

Словоформы с максимальным количеством грамматических разборов:
США: 6 вариантов разбора
кино: 5 вариантов разбора
евро: 5 вариантов разбора
компании: 5 вариантов разбора
какой: 5 вариантов разбора
одной: 4 вариантов разбора
своей: 4 вариантов разбора
лица: 4 вариантов разбора
этой: 4 вариантов разбора
версии: 4 вариантов разбора


## Задание 3
Загрузите один из файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуры через DependencyGraph, выберите 3 любых отношения и для каждого найдите топ-5 самых встречаемых пар слов, связанных этим отношением.

Для самой частотной пары слов в каждом из отношений вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)` и сортируя результат по порядку слов в предложениях, аналогично тому как я делал с summaries только у вас будет два слова)
В итоге у вас должен получится что-то такое:

```
### отношение
relation_name

### топ 5 пар слов связанных этим отношением
(word1, word2), (word3, word4), (word5, word6), (word7, word8), (word9, word10)

### подзависимые для самого частотного
(subword word1 subword, word2 subword subword)

... (и так три раза)
```
